# Determinar la nota final de los estudiantes

En principio se intentaba determinar la nota dependiendo únicamente de las características sociales.
Tras el estudio se vió que salvo unas características muy específicas la correlacionalidad no era lo suficientemente fuerte para determinar la nota, por lo que se integró la nota de escritura y lectura para poder determinar la de matemáticas por su fuerte correlación. Se estudian modelos de árboles de decisión y modelos lineales para determinar cual es el modelo más óptimo. Cuando se utilizaban los datos únicamente sociales se veía una fuerte linealidad en los datos por lo que se entiende que seguramente un modelo lineal será el más adecuado para este tipo de datos.

OBTENER LOS DATOS:

* El dataset que estoy utilizando pertenece a Kaggle y permite su uso, por lo que no necesito de ningún permiso adicional.

In [1]:
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el CSV desde la ruta Github
url="https://raw.githubusercontent.com/Aiadevop/math_student_grade/refs/heads/main/data/processed/df_tree.csv"
df = pd.read_csv(url)

# Mostrar las primeras filas
df.head(10)

,parental_level_of_education_some_high_school,parental_level_of_education_high_school,parental_level_of_education_some_college,parental_level_of_education_bachelors_degree,parental_level_of_education_masters_degree,gender,lunch,test_preparation_course,math_score,reading_score,writing_score,race_ethnicity_group_B,race_ethnicity_group_C,race_ethnicity_group_D,race_ethnicity_group_E
0,0.0,0.0,0.0,1.0,0.0,0,1,0,72,72,74,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0,1,1,69,90,88,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0,1,0,90,95,93,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1,0,0,47,57,44,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1,1,0,76,78,75,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0,1,0,71,83,78,1.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,0,1,1,88,95,92,1.0,0.0,0.0,0.0
7,0.0,0.0,1.0,0.0,0.0,1,0,0,40,43,39,1.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,1,0,1,64,64,67,0.0,0.0,1.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0,0,0,38,60,50,1.0,0.0,0.0,0.0


In [2]:
from sklearn.model_selection import train_test_split

# Para determinar si el escalado de las variables está justificado voy a entrenar el modelo sin el escalado.
df_dt = df.copy()

# 1️⃣ Separar X (features) e y (target)
X = df_dt.drop(['math_score'], axis=1)
y = df_dt['math_score']

# 2️⃣ Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# 3️⃣ Verificar tamaños y distribución
print(f"Tamaño X_train: {X_train.shape}")
print(f"Tamaño X_test: {X_test.shape}")

print("\nDistribución y_train:")
print(y_train.describe())

print("\nDistribución y_test:")
print(y_test.describe())

Tamaño X_train: (800, 14)
Tamaño X_test: (200, 14)

Distribución y_train:
count    800.000000
mean      66.496250
std       15.024302
min        8.000000
25%       57.000000
50%       66.000000
75%       77.000000
max      100.000000
Name: math_score, dtype: float64

Distribución y_test:
count    200.000000
mean      64.460000
std       15.638453
min        0.000000
25%       55.000000
50%       66.000000
75%       76.000000
max      100.000000
Name: math_score, dtype: float64


In [3]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Diccionario con los modelos basados en árboles
modelos_arboles = {
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    "HistGradientBoosting": HistGradientBoostingRegressor(max_iter=100, learning_rate=0.1, random_state=42)
}

# DataFrame para guardar resultados
resultados_arboles = []

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos_arboles.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    resultados_arboles.append({
        "Modelo": nombre,
        "MAE": mae,
        "RMSE": rmse,
        "R²": r2
    })

# Convertir a DataFrame para comparar
df_resultados_arboles = pd.DataFrame(resultados_arboles).sort_values(by="R²", ascending=False)

print("🌳 Resultados de los modelos basados en árboles:")
display(df_resultados_arboles)


🌳 Resultados de los modelos basados en árboles:


,Modelo,MAE,RMSE,R²
2,GradientBoosting,4.290004,5.564008,0.872777
1,RandomForest,4.730900,6.059799,0.849094
3,HistGradientBoosting,4.663080,6.172309,0.843439
0,DecisionTree,6.505000,8.251364,0.720204


In [ ]:
# El rendimiento base de este modelo ya es menos óptimo que los de regresión lineal, aunque GradientBoosting se acerca.
# Se decide utilizar modelos de regresión lineal